In [14]:
import math
import numpy as np
import pandas as pd

np.random.seed(17)

k = 3
idxs = None
# idxs = [78, 42, 102]

In [15]:
data = pd.read_csv("./iris.csv")
data.head()

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [16]:
X = data.iloc[:, :4].values
# y = data.iloc[:, 4].values
X[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

### KMeans - Marcio

In [17]:
class MarcioKMeans:
    def __init__(self, k=2, tolerance=0.001, max_iter=100):
        self.k = k
        self.max_iterations = max_iter
        self.tolerance = tolerance

    def euclidean_distance(self, point1, point2):
        return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

    def predict(self, point):
        """
            É realizado o calculo das distâncias entre um determinado ponto
            e cada um dos centróides definidos pelo agrupamento. O cluster que
            resultar na menor distância sera o resultado da predição

            obs: A função np.linalg.norm() retorna a distância euclidiana,
                 o uso desta função é mais eficiente para este calculo
        """
        distances = [np.linalg.norm(point - self.centroids[centroid]) for centroid in self.centroids]
        classification = distances.index(min(distances))
        return classification

    def fit(self, data, idxs=None):
        """
            A inicialização dos centróides é feita de maneira simples:
            * São escolhidos K centróides aleatoriamente do dataset.
        """
        if idxs is None:
            print("Gerando Indices aleatorios...")
            idxs = np.random.choice(len(data), self.k, replace=False)

        self.centroids = {i: data[idx] for i, idx in enumerate(idxs)}
        print(f"Initial centroids {idxs}:")
        print(np.array(list(self.centroids.values())))

        for i in range(self.max_iterations):
            self.classes = {j: [] for j in range(self.k)}  # Inicialização do dicionário de classes

            '''  
                Sera computada as distancias euclidianas de um determinado 
                ponto i com todos os centróides definidos. Após o preenchimento
                da lista de distâncias, será definido o cluster_index, que 
                representa o índice do cluster que obteve a menor distância 
                euclidiana com o ponto (O mesmo cluster_index também será 
                utilizado como index da 'classe' para definir para qual cluster
                o ponto será alocado).  

                Obs: O mesmo processo será repetido para todos os pontos do dataset

            '''
            for point in data:
                distances = []
                for index in self.centroids:
                    distances.append(self.euclidean_distance(point, self.centroids[index]))
                cluster_index = distances.index(min(distances))
                self.classes[cluster_index].append(point)
            
            '''  
                Previous é um dicionário python com base nos centróides, que 
                armazena os valores prévios dos centróides definidos
            '''
            previous = self.centroids

            ''' 
                Para cada índice de cluster dentro do dicionário de classes
                é calculado um novo valor de centróide, levando em consideraço a média 
                dos valores dos pontos que foram alocados do dicionário 'classes' no 
                trecho de código anterior: self.classes[cluster_index].append(point)

                obs: A flag is_optimal é definida como True
            '''
            for cluster_index in self.classes:
                self.centroids[cluster_index] = np.average(self.classes[cluster_index], axis=0)
            is_optimal = True

            '''  
                Para cada centroid definido, será avaliada o qual foi a variação
                de valor do novo centróide calculado. Se a porcentagem da varição
                for superior à tolerância estabelecida previamente, o loop de execução
                continua, caso contrário (is_optimal is True), a execução é encerrada. 
            '''
            for centroid in self.centroids:
                original_centroid = previous[centroid]
                curr = self.centroids[centroid]
                if np.sum((curr - original_centroid) / original_centroid * 100.0) > self.tolerance:
                    is_optimal = False

            if is_optimal:
                break


m_kmeans = MarcioKMeans(k=k)
m_kmeans.fit(X, idxs)
m_classes = [m_kmeans.predict(x) for x in X]

Gerando Indices aleatorios...
Initial centroids [ 16  78 145]:
[[5.4 3.9 1.3 0.4]
 [6.  2.9 4.5 1.5]
 [6.7 3.  5.2 2.3]]


In [21]:
### KMeans - Andrey

In [22]:
def a_kmeans(X: np.ndarray, k: int, idxs=None):
    if not idxs:
        idxs = np.random.choice(X.shape[0], k, replace=False)

    centroids = X[idxs]
    print(f"Initial centroids {idxs}:")
    print(centroids)
    labels = np.zeros(X.shape[0])

    converged = False
    while not converged:
        # update cluster membership
        distances = np.linalg.norm(X[:, None] - centroids, axis=2)
        labels = np.argmin(distances, axis=1)  # closest

        # update centroids
        new_centroids = np.array([np.mean(X[labels == i], axis=0) for i in range(k)])

        if np.all(centroids == new_centroids):
            converged = True

        centroids = new_centroids.copy()

    return labels, centroids

a_classes, a_classes = a_kmeans(X, k, idxs)

Initial centroids [121  26  96]:
[[5.6 2.8 4.9 2. ]
 [5.  3.4 1.6 0.4]
 [5.7 2.9 4.2 1.3]]


In [23]:
output = data.copy()
output["marcio"] = m_classes
output["andrey"] = m_classes
output.to_csv("result.csv")